# Praktikum Pengantar Pembelajaran Mesin


---
## Bab 5. Klasifikasi Decision Tree


### 1) Import Data

Praktikum kali ini menggunakan dataset [Car Evaluation Dataset](https://archive.ics.uci.edu/ml/datasets/Car+Evaluation) dari UCI Machine Learning Repository. Dataset ini telah digunakan pada praktikum sebelumnya. Detail fitur dapat Anda pelajari pada link yang tersedia.

Unduh dataset yang akan digunakan pada praktikum kali ini. Anda dapat menggunakan aplikasi wget untuk mendowload dataset dan menyimpannya dalam Google Colab. Jalankan cell di bawah ini untuk mengunduh dataset

In [1]:
import numpy as np
import pandas as pd

In [2]:
!wget https://gist.githubusercontent.com/Arceister/a890983a3c3f151eebf5702d870d93a7/raw/a4ed5bf0d779a8c8b8cda7145c44e1a36fe8f7ca/car_evaluation.csv

--2024-04-15 04:14:31--  https://gist.githubusercontent.com/Arceister/a890983a3c3f151eebf5702d870d93a7/raw/a4ed5bf0d779a8c8b8cda7145c44e1a36fe8f7ca/car_evaluation.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 51915 (51K) [text/plain]
Saving to: ‘car_evaluation.csv’

car_evaluation.csv  100%[===================>]  50.70K  --.-KB/s    in 0.01s   

2024-04-15 04:14:32 (3.39 MB/s) - ‘car_evaluation.csv’ saved [51915/51915]



Setelah dataset berhasil diunduh, langkah berikutnya adalah membaca dataset dengan memanfaatkan fungsi **readcsv** dari library pandas. Lakukan pembacaan berkas csv ke dalam dataframe dengan nama **data** menggunakan fungsi **readcsv**. Jangan lupa untuk melakukan import library pandas terlebih dahulu


In [3]:
data = pd.read_csv('car_evaluation.csv')



Cek isi dataset Anda dengan menggunakan perintah **head()**

In [4]:
data.head()

,buying,maint,doors,persons,lug_boot,safety,class
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


## 2) Membagi data menjadi data latih dan data uji

Metode pembelajaran mesin memerlukan dua jenis data :


1.   Data latih : Digunakan untuk proses training metode klasifikasi
2.   Data uji : Digunakan untuk proses evaluasi metode klasifikasi

Data uji dan data latih perlu dibuat terpisah (mutualy exclusive) agar hasil evaluasi lebih akurat.

Data uji dan data latih dapat dibuat dengan cara membagi dataset dengan rasio tertentu, misalnya 80% data latih dan 20% data uji.

Library Scikit-learn memiliki fungsi [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) pada modul **model_selection** untuk membagi dataset menjadi data latih dan data uji. Bagilah dataset anda menjadi dua, yaitu **data_latih** dan **data_uji**. Agar pengacakan data dilakukan secara konstan, parameter **random_state** diisi dengan nilai integer tertentu, **pada praktikum ini diset 101**. Kemudian, nilai indeks pada data latih dan data uji diatur ulang agar berurutan nilainya


In [5]:
from sklearn.model_selection import train_test_split
data_latih, data_uji = train_test_split(data, test_size = 0.2 ,random_state = 101)
data_latih.reset_index(drop=True)
data_uji.reset_index(drop=True)

,buying,maint,doors,persons,lug_boot,safety,class
0,vhigh,high,4,more,med,med,unacc
1,low,med,2,more,small,high,unacc
2,vhigh,low,5more,2,big,med,unacc
3,low,vhigh,3,more,small,med,unacc
4,med,low,3,more,small,low,unacc
...,...,...,...,...,...,...,...
341,high,med,2,more,med,med,unacc
342,low,med,4,more,big,med,good
343,vhigh,vhigh,3,2,big,med,unacc
344,vhigh,low,2,4,med,low,unacc


Tampilkan banyaknya data pada **data_latih** dan **data_uji**. Seharusnya **data_latih** terdiri dari 208 data, dan **data_uji** terdiri dari 52 data

In [6]:
print(data_latih.shape[0])
print(data_uji.shape[0])

1382
346


## 3) Menghitung Gini

Nilai Gini merupakan salah satu kriteria penentu variabel apa yang akan digunakan untuk membentuk cabang pada decision tree. Variabel dengan nilai Gini terbesar akan digunakan sebagai pembentukan cabang

Buatlah fungsi bernama **hitung_gini** yang berfungsi menghitung nilai Gini dari suatu nilai pada sebuah variabel

In [7]:
def hitung_gini(kolom_kelas):
  # refers to elemen and banyak of class/label
  elemen,banyak = np.unique(kolom_kelas,return_counts = True)
  nilai_gini = 1 - np.sum([(banyak[i]/np.sum(banyak))**2 for i in range(len(elemen))])
  return nilai_gini

In [8]:
kelas, jumlah = np.unique(data_latih['buying'], return_counts = True)
print("Kelas:", kelas)
print("Jumlah: ", jumlah)
print(np.sum(jumlah))

Kelas: ['high' 'low' 'med' 'vhigh']
Jumlah:  [338 358 346 340]
1382


In [9]:
kolom_kelas = data_latih.where(data_latih['buying']=='high').dropna()['class']
print(kolom_kelas.value_counts())
print(np.sum(kolom_kelas.value_counts()))

class
unacc    254
acc       84
Name: count, dtype: int64
338


In [10]:
gini = hitung_gini(kolom_kelas)
gini

0.37351633346171353

In [11]:
print((84/338)**2)
print((254/338)**2)
print(1-((84/338)**2+(254/338)**2))

0.06176254332831483
0.5647211232099716
0.37351633346171353


Buatlah fungsi bernama **gini_split** yang digunakan untuk menghitung nilai Gini keseluruhan dari sebuah variabel.

In [12]:
def gini_split(data,nama_fitur_split,nama_fitur_kelas):
  # refers to nilai and banyak from fitur not class
    nilai,banyak= np.unique(data[nama_fitur_split],return_counts=True)
    gini_split = np.sum([(banyak[i]/np.sum(banyak))*hitung_gini(data.where(data[nama_fitur_split]==nilai[i]).dropna()[nama_fitur_kelas]) for i in range(len(nilai))])
    return gini_split

In [13]:
#Test Output
def gini_split_output(data,nama_fitur_split,nama_fitur_kelas):
  # refers to nilai and banyak from fitur not class
    nilai,banyak= np.unique(data[nama_fitur_split],return_counts=True)
    print(nilai)
    print(banyak,"\n")
    for i in range(len(nilai)):
      print(f"Nilai dari {nilai[i]} adalah {banyak[i]}")
      print(f"Didalam {nilai[i]} terdapat \n{(data.where(data[nama_fitur_split]==nilai[i]).dropna()[nama_fitur_kelas]).value_counts()} ")
      print("Nilai Sum : ",np.sum(banyak))
      print(f"Nilai dari gini dari {nilai[i]} : {hitung_gini(data.where(data[nama_fitur_split]==nilai[i]).dropna()[nama_fitur_kelas])}")
      print(f"Nilai dari gini split {nilai[i]} adalah {(banyak[i]/np.sum(banyak))*hitung_gini(data.where(data[nama_fitur_split]==nilai[i]).dropna()[nama_fitur_kelas])}")
      print("")
    nilai_akhir = np.sum([(banyak[i]/np.sum(banyak))*hitung_gini(data.where(data[nama_fitur_split]==nilai[i]).dropna()[nama_fitur_kelas]) for i in range(len(nilai))])
    print(f"Nilai Akhir dari gini {nama_fitur_split} adalah {nilai_akhir}")

In [14]:
gini_split_output(data_latih,"buying","class")

['high' 'low' 'med' 'vhigh']
[338 358 346 340] 

Nilai dari high adalah 338
Didalam high terdapat 
class
unacc    254
acc       84
Name: count, dtype: int64 
Nilai Sum :  1382
Nilai dari gini dari high : 0.37351633346171353
Nilai dari gini split high adalah 0.0913520410347751

Nilai dari low adalah 358
Didalam low terdapat 
class
unacc    215
acc       70
good      38
vgood     35
Name: count, dtype: int64 
Nilai Sum :  1382
Nilai dari gini dari low : 0.5802721513061391
Nilai dari gini split low adalah 0.15031651965817497

Nilai dari med adalah 346
Didalam med terdapat 
class
unacc    209
acc      100
vgood     19
good      18
Name: count, dtype: int64 
Nilai Sum :  1382
Nilai dari gini dari med : 0.5458752380634169
Nilai dari gini split med adalah 0.13666630417506676

Nilai dari vhigh adalah 340
Didalam vhigh terdapat 
class
unacc    280
acc       60
Name: count, dtype: int64 
Nilai Sum :  1382
Nilai dari gini dari vhigh : 0.2906574394463668
Nilai dari gini split vhigh adalah 0.071507

Ujilah fungsi **gini_split** menggunakan data_latih pada variabel **buying** dan variabel kelas bernama **class**.

In [15]:
gini_split(data_latih,"buying","class")

0.4498424838345615

## 4) Pembentukan pohon

Pembentukan pohon dilakukan secara rekursif. Seperti metode rekursif pada umumnya, perlu ditentukan kondisi berhenti terlebih dahulu. Kondisi berhenti pada pembentukan pohon adalah:


1.   Jika hanya ada satu kelas pada data, kembalikan kelas tersebut
2.   Jika fitur data  = 0 (tidak ada fitur yang tersisa), kembalikan kelas dari parent
3. Jika data kosong (tidak ada data), kembalikan kelas dengan frekuensi terbanyak

Selain kondisi berhenti tersebut, dilakukan pembentukan pohon secara rekursif menggunakan fungsi **buat_tree**.



In [16]:
np.unique(data["class"])[np.argmax(np.unique(data["class"],return_counts=True)[1])]

'unacc'

In [17]:
np.unique(data["class"])[np.argmin(np.unique(data["class"],return_counts=True)[1])]

'vgood'

In [18]:
jumlah = np.unique(data["class"], return_counts=True)[1]
# np.argmin dan argmax mengembalikan nilai index dari nilai yang dicari
np.argmin(jumlah)

3

In [19]:
def buat_tree(data,data_awal, daftar_fitur, nama_fitur_kelas,kelas_parent_node=None):
  #jika hanya ada satu kelas pada data
  if len(np.unique(data[nama_fitur_kelas])) <= 1:
    return np.unique(data[nama_fitur_kelas])[0]
  #jika data kosong
  elif len(data)==0:
    return np.unique(data_awal[nama_fitur_kelas])[np.argmax(np.unique(data_awal[nama_fitur_kelas],return_counts=True)[1])]
  #jika tidak ada fitur yang terisa
  elif len(daftar_fitur) ==0:
    return kelas_parent_node
  else:
    kelas_parent_node = np.unique(data[nama_fitur_kelas])[np.argmax(np.unique(data[nama_fitur_kelas],return_counts=True)[1])]
    # Hitung nilai gini
    nilai_split = [gini_split(data,fitur,nama_fitur_kelas) for fitur in daftar_fitur]
    index_fitur_terbaik = np.argmin(nilai_split)
    fitur_terbaik = daftar_fitur[index_fitur_terbaik]

    tree = {fitur_terbaik:{}}

    daftar_fitur = [i for i in daftar_fitur if i != fitur_terbaik]

    for nilai in np.unique(data[fitur_terbaik]): #looping nilai dari fitur
      sub_data = data.where(data[fitur_terbaik] == nilai).dropna()
      # Recursive
      subtree = buat_tree(sub_data,data_awal,daftar_fitur,nama_fitur_kelas,kelas_parent_node)
      tree[fitur_terbaik][nilai]=subtree

  return(tree)

Buatlah tree menggunakan data latih yang tersedia

In [20]:
tree = buat_tree(data_latih,data_latih,data_latih.columns[:-1],'class')

Tampilkan tree yang terbentuk. Gunakan library **pprint** untuk menampilkan dictionary secara teratur.

In [21]:
from pprint import pprint
pprint(tree)

{'safety': {'high': {'persons': {'2': 'unacc',
                                 '4': {'buying': {'high': {'maint': {'high': 'acc',
                                                                     'low': 'acc',
                                                                     'med': 'acc',
                                                                     'vhigh': 'unacc'}},
                                                  'low': {'maint': {'high': {'lug_boot': {'big': 'vgood',
                                                                                          'med': {'doors': {'2': 'acc',
                                                                                                            '3': 'acc',
                                                                                                            '4': 'vgood'}},
                                                                                          'small': 'acc'}},
                                    

## 5) Proses prediksi

Proses prediksi kelas pada data uji dilakukan dengan melakukan *tree traversal* sampai menemui leaf.

In [22]:
def prediksi(data_uji,tree):
  for key in list(data_uji.keys()):
    if key in list(tree.keys()):
      try:
        hasil = tree[key][data_uji[key]]
      except:
        return 1
      hasil = tree[key][data_uji[key]]
      if isinstance(hasil,dict):
        return prediksi(data_uji,hasil)
      else:
        return hasil

## 6) Proses Pengujian
Lakukan pengujian menggunakan data uji. Kelas pada data uji perlu dihapus dan data uji perlu diubah menjadi dictionary

In [23]:
data_uji_dict = data_uji.iloc[:,:-1].to_dict(orient = "records")

Lakukan pengujian terhadap keseluruhan data uji menggunakan looping.

In [24]:
hasil_prediksi_total = []
for i in range(len(data_uji_dict)):
  hasil_prediksi = prediksi(data_uji_dict[i],tree)
  hasil_prediksi_total.append(hasil_prediksi)

Bandingkan hasil prediksi dengan label sebenarnya. Hitunglah banyaknya data uji yang memiliki kelas prediksi sama dengan kelas sebenarnya

In [25]:
print("Total data uji: ", data_uji.shape[0])
print("Total prediksi benar: ",sum(hasil_prediksi_total==data_uji['class']))

Total data uji:  346
Total prediksi benar:  298


## TUGAS
Pada tugas kali ini Anda diminta memodifikasi metode pembentukan tree yang telah Anda agar metode tersebut menggunakan information gain sebagai dasar percabangan. Lengkapilah kerangka source code di bawah ini

Lengkapi fungsi hitung_entropy

In [26]:
def hitung_entropy(kolom_kelas):
  elemen, banyak = np.unique(kolom_kelas, return_counts=True)
  entropy = -np.sum([(banyak[i]/np.sum(banyak))*np.log2(banyak[i]/np.sum(banyak)) for i in range(len(elemen))])
  return entropy

Lengkapi fungsi information_gain

In [27]:
def information_gain(data, nama_fitur_split, nama_fitur_kelas):
  nilai, banyak = np.unique(data[nama_fitur_split], return_counts=True)
  entropy_awal = hitung_entropy(data[nama_fitur_kelas])
  entropy_split = np.sum([(banyak[i]/np.sum(banyak))*hitung_entropy(data.where(data[nama_fitur_split]==nilai[i]).dropna()[nama_fitur_kelas]) for i in range(len(nilai))])
  information_gain = entropy_awal - entropy_split
  return information_gain


Lengkapi fungsi **buat_tree_ig**. Isinya sama persis dengan fungsi **buat_tree**, hanya saja penghitungan **gini_split** diganti dengan **information_gain**. Selain itu, percabangan dilakukan dengan menggunakan nilai **information_gain** **terbesar**

In [28]:
def buat_tree_ig(data, data_awal, daftar_fitur, nama_fitur_kelas, kelas_parent_node=None):
    # jika hanya ada satu kelas pada data
    if len(np.unique(data[nama_fitur_kelas])) <= 1:
        return np.unique(data[nama_fitur_kelas])[0]
    # jika data kosong
    elif len(data) == 0:
        return np.unique(data_awal[nama_fitur_kelas])[np.argmax(np.unique(data_awal[nama_fitur_kelas], return_counts=True)[1])]
    # jika tidak ada fitur yang tersisa
    elif len(daftar_fitur) == 0:
        return kelas_parent_node
    else:
        kelas_parent_node = np.unique(data[nama_fitur_kelas])[np.argmax(np.unique(data[nama_fitur_kelas], return_counts=True)[1])]
        nilai_ig = [information_gain(data, fitur, nama_fitur_kelas) for fitur in daftar_fitur]
        index_fitur_terbaik = np.argmax(nilai_ig)
        fitur_terbaik = daftar_fitur[index_fitur_terbaik]
        tree = {fitur_terbaik:{}}
        daftar_fitur = [i for i in daftar_fitur if i != fitur_terbaik]
        for nilai in np.unique(data[fitur_terbaik]):
            sub_data = data.where(data[fitur_terbaik] == nilai).dropna()
            subtree = buat_tree_ig(sub_data, data_awal, daftar_fitur, nama_fitur_kelas, kelas_parent_node)
            tree[fitur_terbaik][nilai] = subtree
        return tree

Lakukan pembentukan tree menggunakan fungsi **buat_tree_ig**

In [29]:
tree_ig = buat_tree_ig(data_latih,data_latih,data_latih.columns[:-1],'class')

Tampilkan tree yang terbentuk

In [30]:
pprint(tree_ig)

{'safety': {'high': {'persons': {'2': 'unacc',
                                 '4': {'buying': {'high': {'maint': {'high': 'acc',
                                                                     'low': 'acc',
                                                                     'med': 'acc',
                                                                     'vhigh': 'unacc'}},
                                                  'low': {'maint': {'high': {'lug_boot': {'big': 'vgood',
                                                                                          'med': {'doors': {'2': 'acc',
                                                                                                            '3': 'acc',
                                                                                                            '4': 'vgood'}},
                                                                                          'small': 'acc'}},
                                    

Lakukan pengujian menggunakan tree yang terbentuk

In [31]:
hasil_prediksi_total_ig = []
for i in range(len(data_uji_dict)):
  hasil_prediksi = prediksi(data_uji_dict[i],tree_ig)
  hasil_prediksi_total_ig.append(hasil_prediksi)
print("Total prediksi benar: ",sum(hasil_prediksi_total_ig==data_uji['class']))

Total prediksi benar:  298


# PERTANYAAN

Jawablah pertanyaan di bawah ini



1.   Amati tree yang dihasilkan dengan kriteria percabangan GINI dan Information Gain. Apa perbedaan tree yang dihasilkan dari kedua metode tersebut?
2.   Apakah penggunaan Information Gain dapat meningkatkan akurasi prediksi?



Tulis jawaban Anda di cell ini


1.   dalam tree dengan metode GINI, fitur pertama yang digunakan untuk melakukan percabangan adalah safety, sedangkan dalam tree dengan metode information gain, fitur pertama yang digunakan adalah persons. selainn itu ada beberapa percabangan yang berbeda. ini karena 2 metode tersebut menggunakan kriteria yang berbeda dalam memilih fitur yang paling baik untuk melakukan percabangan, GINI dengan menghitung ketidakmurnian kelas dan information gain dengan menghitung keuntungan informasi.
2.   tidak dapat dipastikan karena pemilihan metode didasarkan pada karakteristik data yang digunakan, tapi dalam beberapa kasus, information gain dapat memberikan hasil yang lebih baik daripada GINI.

